#Q5

In [27]:
import numpy as np 
import pandas as pd 	
import matplotlib.pyplot as plt 
import math
dataset = pd.read_csv('penguins.csv')

##cleaning data & Importing the dataset


In [28]:
row_deleted =[]
for row in range(dataset.shape[0]):
   for column in range(dataset.shape[1]):
      if(dataset.iloc[row,column] == "x"):
          row_deleted.append(row)

row_deleted = [*set(row_deleted)]
for row in row_deleted:
  dataset = dataset.drop(row)
X = dataset.iloc[ : ,1 :].values
y = dataset.iloc[ : , 0].values
print(X)
print(y)


[['39.1' '18.7' '181' '3750']
 ['39.5' '17.4' '186' '3800']
 ['40.3' '18' '195' '3250']
 ...
 ['50.4' '15.7' '222' '5750']
 ['45.2' '14.8' '212' '5200']
 ['49.9' '16.1' '213' '5400']]
['Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Ad

## Splitting the dataset into the Training set and Test set

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

##GET probibility

In [31]:
unique_vals, counts = np.unique(y_train, return_counts=True)
for val, count in zip(unique_vals, counts):
    probability_=count/len(y_train)
    print(f"{val}: {count}    probability : {probability_}")

Adelie: 109    probability : 0.43253968253968256
Chinstrap: 54    probability : 0.21428571428571427
Gentoo: 89    probability : 0.3531746031746032


## Training the naïve bayes

In [37]:
from scipy.stats import norm
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score, recall_score
import math
def probabolity_of_specific_kind(mean_class_1_x,var_class_1_x,mean_class_all_x,
                                 var_class_all_x,prior_probility_1_x,prior_probility_all_x):
  final_probility_1_x_list=[]
  final_probility_all_x_list=[]
  for i in range(len(y_test)):
    final_probility_1_x=prior_probility_1_x
    final_probility_all_x=prior_probility_all_x
    for j in range(X_test.shape[1]):
      final_probility_1_x =final_probility_1_x * norm.pdf(
           X_test[i][j], mean_class_1_x[j], math.sqrt(var_class_1_x[j]))
      final_probility_all_x = final_probility_all_x * final_probility_all_x * norm.pdf(
           X_test[i][j], mean_class_all_x[j], math.sqrt(var_class_all_x[j]))
    final_probility_1_x_list.append(final_probility_1_x)
    final_probility_all_x_list.append(final_probility_all_x)
  return(final_probility_1_x_list,final_probility_all_x_list)




def Naive_Bayes_1_vs_all(our_class):
  class_1_x =[]
  class_all_x =[]

  for i in range(len(y_train)):
    if(y_train[i]==our_class):
      class_1_x.append(X_train[i])
    else:
      class_all_x.append(X_train[i])
  prior_probility_1_x = len(class_1_x)/len(y_train)
  prior_probility_all_x = len(class_all_x)/len(y_train)
  #convert to ndarray
  class_1_x =np.array(class_1_x)
  class_all_x =np.array(class_all_x)
  #caculate mean and var row by row
  mean_class_1_x = np.mean(class_1_x, axis=0)
  var_class_1_x = np.var(class_1_x, axis=0)
  mean_class_all_x = np.mean(class_all_x, axis=0)
  var_class_all_x = np.var(class_all_x, axis=0)
  #caculate probibility
  return probabolity_of_specific_kind(mean_class_1_x,var_class_1_x,
                               mean_class_all_x,var_class_all_x,prior_probility_1_x,prior_probility_all_x)

def caculate_probability():
    probabilities_of_x = []
    probabilities_of_all = []
    for i in unique_vals:
      list_x= []
      list_all = []
      list_x,list_all = Naive_Bayes_1_vs_all(i)
      probabilities_of_x.append(list_x)
      probabilities_of_all.append(list_all)
    finall_decision=[]
    for i in range(len(y_test)):
      probability_list =[probabilities_of_x[0][i],probabilities_of_x[1][i],probabilities_of_x[2][i]]
      max_probability = max(probability_list)
      finall_decision.append(unique_vals[probability_list.index(max_probability)])
    # print(finall_decision)
    # print(len(finall_decision))
    correct =0
    for i in range(len(finall_decision)):
      if(finall_decision[i]==y_test[i]):
        correct +=1
    print(confusion_matrix(y_test, finall_decision))
    print("accuracy_score = ",accuracy_score(y_test, finall_decision))
    print("precision_score = ",precision_score(y_test, finall_decision, average="macro"))
    print( "recall_score = ",recall_score(y_test, finall_decision, average="macro"))
caculate_probability()

[[41  0  0]
 [ 1 12  0]
 [ 0  0 31]]
accuracy_score =  0.9882352941176471
precision_score =  0.9920634920634921
recall_score =  0.9743589743589745


## Training the Naive Bayes model on the Training set

In [33]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [34]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Chinstrap' 'Chinstrap']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Chinstrap' 'Chinstrap']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Chinstrap' 'Chinstrap']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Chinstrap' 'Chinstrap']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Chinstrap']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Chinstrap' 'Chinstrap']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 ['Gentoo' 'Gentoo']
 ['Adelie' 'Adelie']
 ['Adelie' 'Adelie']
 

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print("accuracy_score = ",accuracy_score(y_test, y_pred))
print("precision_score = ",precision_score(y_test, y_pred, average="macro"))
print( "recall_score = ",recall_score(y_test, y_pred, average="macro"))

[[41  0  0]
 [ 1 12  0]
 [ 0  0 31]]
accuracy_score =  0.9882352941176471
precision_score =  0.9920634920634921
recall_score =  0.9743589743589745
